# MosquitoAlert - YoloV5 Baseline Submission

![](https://images.aicrowd.com/raw_images/challenges/social_media_image_file/1124/1c83e8678106d51ab51f.png)

This notebook will contains instructions and example submission with yolov5 trained on the dataset.

## Installations 🤖

1. `aicrowd-cli` for downloading challenge data and making submissions
2. `pyarrow` for saving to parquet for submissions

In [1]:
!pip install -U ultralytics tqdm opencv-python torchvision pandas Pillow gdown aicrowd-cli

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.6/612.6 kB 10.3 MB/s eta 0:00:009.1 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━ 44.2/61.7 MB 11.5 MB/s eta 0:00:02^C
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━ 44.4/61.7 MB 11.5 MB/s eta 0:00:02
ERROR: Operation cancelled by user


In [2]:
import torch
import os
from pathlib import Path
import cv2
from timeit import default_timer as timer
import pandas as pd
from tqdm.auto import tqdm
from PIL import Image

# Download Dataset and Models

In [3]:
!aicrowd login

/bin/bash: line 1: aicrowd: command not found


In [4]:
!aicrowd dataset download --challenge mosquitoalert-challenge-2023

/bin/bash: line 1: aicrowd: command not found


In [5]:
!unzip -qq test_images_phase1.zip -d test_images_phase1/

unzip:  cannot find or open test_images_phase1.zip, test_images_phase1.zip.zip or test_images_phase1.zip.ZIP.


In [12]:
# Downloading the weight file at https://drive.google.com/file/d/1s7EoK3V9YTwQOxhuO1ERABRZ95atz8PS/view
!gdown "1s7EoK3V9YTwQOxhuO1ERABRZ95atz8PS"

Downloading...
From: https://drive.google.com/uc?id=1s7EoK3V9YTwQOxhuO1ERABRZ95atz8PS
To: /content/mosquitoalert-yolov5-baseline.pt
100% 14.4M/14.4M [00:00<00:00, 75.9MB/s]


In [4]:
TEST_DATA_DIR = 'test_images_phase1/'

In [5]:
def classify_image(image):
    image_information = {}
    result = yolov5_model(image)
    result_df = result.pandas().xyxy[0]
    if result_df.empty:
        print('No results from yolov5 model!')
    else:
        image_information = result_df.to_dict()
    return image_information

# getting mosquito_class name from predicted result
def extract_predicted_mosquito_class_name(extractedInformation):
    mosquito_class = ""
    if extractedInformation is not None:
        mosquito_class = str(extractedInformation.get("name").get(0))
    return mosquito_class

# getting mosquito_class number from predicted result
def extract_predicted_mosquito_class_number(extractedInformation):
    mosquito_class = ""
    if extractedInformation is not None:
        mosquito_class = str(extractedInformation.get("class").get(0))
    return mosquito_class

# getting mosquito_class confidence score from predicted result
def extract_predicted_mosquito_class_confidence(extractedInformation):
    mosquito_class = ""
    if extractedInformation is not None:
        mosquito_class = str(extractedInformation.get("confidence").get(0))
    return mosquito_class

# getting mosquito bounding box from predicted result
# it looks like the results are in different notation
# than the uoutput file
# Pascal VOC? (top, left, bottom, right)?
def extract_predicted_mosquito_bbox(extractedInformation):
    bbox = []
    if extractedInformation is not None:
        xmin = str(extractedInformation.get("xmin").get(0))
        ymin = str(extractedInformation.get("ymin").get(0))
        xmax = str(extractedInformation.get("xmax").get(0))
        ymax = str(extractedInformation.get("ymax").get(0))
        bbox = [xmin, ymin, xmax, ymax]
    return bbox

# find image id
def find_image_id(original_image):
    image_name = os.path.splitext(original_image)[0]
    return image_name

In [6]:
# path to dataset
root_images = os.path.join(TEST_DATA_DIR)

all_images = os.listdir(root_images)
print(f"Total images: {len(all_images)}")

class_labels = {
    "aegypti":      0,
    "albopictus":   1,
    "anopheles":    2,
    "culex":        3,
    "culiseta":     4,
    "japonicus/koreicus": 5
}

# counter for correctly recognized mosquito classes
counter = 0
labels_counter = 0
no_mosquito_detected = 0
rows = []

# yolov5 challenge trained baseline
trained_model_pretrained = "mosquitoalert-yolov5-baseline.pt"

# yolov5_model
# Colab gets an error on this -> Use Runtime -> Restart Session (The data will not be deleted)
yolov5_model = torch.hub.load('ultralytics/yolov5', 'custom', path = trained_model_pretrained, force_reload = True)

for original_image in tqdm(all_images):
    try:
        original_image_file = os.path.join(root_images, original_image)
        # checking if it is a file
        if os.path.isfile(original_image_file):
            # opening testing image
            # print(f'You are watching: {original_image}')
            # classifying image by yolov5 model
            predictedInformation = classify_image(original_image_file)
            mosquito_class_name_predicted = ""
            mosquito_class_number_predicted = ""
            mosquito_class_confidence = 0
            mosquito_class_bbox = [0, 0, 0, 0]

            if predictedInformation:
                mosquito_class_name_predicted = extract_predicted_mosquito_class_name(predictedInformation)
                mosquito_class_number_predicted = extract_predicted_mosquito_class_number(predictedInformation)
                mosquito_class_confidence = extract_predicted_mosquito_class_confidence(predictedInformation)
                mosquito_class_bbox = extract_predicted_mosquito_bbox(predictedInformation)
                counter += 1
            else:
                no_mosquito_detected += 1
            # print(f"Predicted mosquito class: {mosquito_class_name_predicted} with {float(mosquito_class_confidence):.2f} confidence score.")
            #  bbox = [xmin, ymin, xmax, ymax]
            # create row for csv file
            row = [original_image, mosquito_class_name_predicted, mosquito_class_number_predicted, mosquito_class_confidence,
                   mosquito_class_bbox[0], mosquito_class_bbox[1], mosquito_class_bbox[2], mosquito_class_bbox[3]]
            rows.append(row)
            labels_counter += 1
            # print(f"Finished file reading, file {original_image} read correctly!")
    except Exception as e:
        print(f'Unable to process file: {original_image}!')
        print(f'Exception: {e}!')

Total images: 2676


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
requirements: Ultralytics requirement "gitpython>=3.1.30" not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: gitpython
    Found existing installation: GitPython 3.1.18
    Uninstalling GitPython-3.1.18:
      Successfully uninstalled GitPython-3.1.18

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2023-6-28 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients
Adding AutoShape... 


  0%|          | 0/2676 [00:00<?, ?it/s]

No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!


In [7]:
df = pd.DataFrame(rows, columns=["img_fName", "class_label", "predicted_class_number", "confidence_score", "bbx_xtl", "bbx_ytl", "bbx_xbr", "bbx_ybr"])
sub_df = df.drop(['predicted_class_number', 'confidence_score'], axis=1, inplace=False)
sub_df.head()

,img_fName,class_label,bbx_xtl,bbx_ytl,bbx_xbr,bbx_ybr
0,751b3f5c-3031-4312-aede-b164889610fc.jpeg,albopictus,246.53277587890625,189.07186889648438,476.0340270996094,413.14471435546875
1,b21d1a3c-f417-41f2-abff-8565fb48f008.jpeg,culex,352.280029296875,375.5829162597656,463.6744689941406,489.3329162597656
2,4d2b9f10-614c-47ed-8126-ed3f0c58cd76.jpeg,albopictus,940.1656494140625,1120.868896484375,1883.7869873046875,1953.6861572265625
3,013aefb4-829a-445c-8b52-59a52badb050.jpeg,japonicus-koreicus,1356.7244873046875,1233.9964599609375,2903.752197265625,2074.76025390625
4,6c3a2882-a266-4d59-b014-904bfbc9c3fd.jpeg,albopictus,392.6656494140625,355.2624816894531,699.4199829101562,625.9489135742188


In [8]:
sub_df.to_csv('submission_phase1.csv', index=False)

In [9]:
!aicrowd submission create -c mosquitoalert-challenge-2023 -f "submission_phase1.csv"

submission_phase1.csv ━━━━╸━━━━━━━━ 35.4% • 114.7/323.6   • 713.3 kB/s • 0:00:01
submission_phase1.csv ━━━━╸━━━━━━━━ 38.0% • 122.9/323.6   • 754.2 kB/s • 0:00:01
submission_phase1.csv ━━━━━╺━━━━━━━ 40.5% • 131.1/323.6   • 797.3 kB/s • 0:00:01
submission_phase1.csv ━━━━━╸━━━━━━━ 43.0% • 139.3/323.6   • 839.2 kB/s • 0:00:01
submission_phase1.csv ━━━━━╸━━━━━━━ 45.6% • 147.5/323.6   • 880.7 kB/s • 0:00:01
submission_phase1.csv ━━━━━━╺━━━━━━ 48.1% • 155.6/323.6   • 921.3 kB/s • 0:00:01
submission_phase1.csv ━━━━━━╸━━━━━━ 50.6% • 163.8/323.6   • 961.1 kB/s • 0:00:01
submission_phase1.csv ━━━━━━╸━━━━━━ 53.2% • 172.0/323.6   • 999.8 kB/s • 0:00:01
submission_phase1.csv ━━━━━━╸━━━━━━ 53.2% • 172.0/323.6   • 999.8 kB/s • 0:00:01
submission_phase1.csv ━━━━━━━╺━━━━━ 55.7% • 180.2/323.6   • 597.4 kB/s • 0:00:01
submission_phase1.csv ━━━━━━━╸━━━━━ 58.2% • 188.4/323.6   • 619.6 kB/s • 0:00:01
submission_phase1.csv ━━━━━━━╸━━━━━ 60.8% • 196.6/323.6   • 641.5 kB/s • 0:00:01
submission_phase1.csv ━━━━━━